In [1]:
from IPython.display import display

from io import StringIO
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
bid_df = pd.read_csv('20201007_bids_cb.csv')
offer_df = pd.read_csv('20201007_da_co.csv')


In [3]:
bid_df.groupby('Type of Bid').first()

,Region,Market Participant Code,Date/Time Beginning (EST),Date/Time End (EST),MW,LMP,Bid ID,PRICE1,MW1,PRICE2,...,PRICE5,MW5,PRICE6,MW6,PRICE7,MW7,PRICE8,MW8,PRICE9,MW9
Type of Bid,,,,,,,,,,,,,,,,,,,,,
D,Central,122062261,10/07/2020 00:00:00,10/07/2020 01:00:00,0.0,15.00,981837552,15.0,205.0,-6.37,...,3.00,0.5,16.19,10.0,15.25,10.0,19.70,0.5,20.52,0.5
F,North,122062236,10/07/2020 00:00:00,10/07/2020 01:00:00,6.0,9.58,122073561,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I,North,122062269,10/07/2020 00:00:00,10/07/2020 01:00:00,15.0,7.99,3936522770,5.0,15.0,17.75,...,2.64,125.0,-85.35,19.0,-77.60,12.0,32.56,0.6,22.27,20.0
P,Central,122062313,10/07/2020 00:00:00,10/07/2020 01:00:00,155.8,10.36,450524077,190.0,155.8,15.11,...,13.55,6.2,12.51,3.1,11.99,3.1,11.47,3.1,NaN,NaN


In [6]:
def validate_bid(row):
    bids = pd.DataFrame({
        'price': row[[('PRICE%d' % i) for i in range(1, 10)]],
        'volume': row[[('MW%d' % i) for i in range(1, 10)]]}
    ).dropna(how='any', axis='index')
    if len(bids) == 0:
        return False
    return bids.price.is_monotonically_increasing() and bids.volume.is_monotonically_decreasing()

In [11]:
bid_df.head(500).loc[bid_df.head(500).apply(validate_bid, axis='columns')].head()

,Region,Market Participant Code,Date/Time Beginning (EST),Date/Time End (EST),MW,LMP,Type of Bid,Bid ID,PRICE1,MW1,...,PRICE5,MW5,PRICE6,MW6,PRICE7,MW7,PRICE8,MW8,PRICE9,MW9


In [14]:
bid_df.iloc[[33]]

,Region,Market Participant Code,Date/Time Beginning (EST),Date/Time End (EST),MW,LMP,Type of Bid,Bid ID,PRICE1,MW1,...,PRICE5,MW5,PRICE6,MW6,PRICE7,MW7,PRICE8,MW8,PRICE9,MW9
33,North,122062236,10/07/2020 08:00:00,10/07/2020 09:00:00,20.0,23.41,F,767746013,NaN,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
